In [36]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel
import gradio as gr

In [37]:
# Set up client
client = OpenAI(api_key=os.getenv('DEEPSEEK_API_KEY'), base_url='https://api.deepseek.com')
model = 'deepseek-chat'

In [ ]:
# Set up class for evaluator to respond
class CityGuide(BaseModel):
    is_acceptable: bool
    feedback: str

In [39]:
# Prompts for main page
def main_system_prompt() -> str:
    prompt = f"You are an Agent on a website that provides information about places asked by the User."
    prompt += f"Your response should be casual, friendly, and inviting for the User to visit that place."
    prompt += f"If you are not sure where that place is, just say so."
    return prompt

In [77]:
# Prompts for evaluator
def evaluator_system_prompt(country: str) -> str:
    prompt = f"You determine if the place mentioned is located in {country}. "
    return prompt

def evaluator_user_prompt(reply, message, history) -> str:
    prompt = f"Here's conversation between the User and Agent: \n\n{history}\n\n"
    prompt += f"Here is the latest message from the User: \n\n{message}\n\n"
    prompt += f"Here is the latest response form the Agent: \n\n{reply}\n\n"
    prompt += f"Please evaluate the response and determine if it is acceptable and give your feedback."
    return prompt

In [ ]:
evaluator = OpenAI(api_key=os.getenv('GOOGLE_API_KEY'), base_url='https://generativelanguage.googleapis.com/v1beta/openai/')

def evaluator_response(country: str, reply, message, history) -> CityGuide:
    messages = [{"role": "system", "content": evaluator_system_prompt(country=country)}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = evaluator.beta.chat.completions.parse(
        messages=messages,
        model='gemini-2.0-flash',
        response_format=CityGuide
    )
    return response.choices[0].message.parsed

In [80]:
# Test evaluator response
response = evaluator_response(country="Thailand", reply="Bangkok is located in Thailand", message="Bangkok", history="")
print(response.is_acceptable)

ParsedChatCompletion[CityGuide](id='vkjnaNuROfCCvr0PpcWBiQc', choices=[ParsedChoice[CityGuide](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[CityGuide](content='{\n  "is_acceptable": true\n}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, parsed=CityGuide(is_acceptable=True)))], created=1759987903, model='gemini-2.0-flash', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=69, total_tokens=80, completion_tokens_details=None, prompt_tokens_details=None))
True


In [87]:
# Main chat function
def chat(message, history):
    system_prompt = main_system_prompt()
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(
        messages=messages,
        model=model
    )
    
    country = "Thailand"
    reply = response.choices[0].message.content

    evaluation = evaluator_response(country=country, reply=reply, message=message, history=history)

    if not evaluation.is_acceptable:
        reply += f"\n\n Too bad it is not in Thailand"
    
    return reply

In [88]:
gr.ChatInterface(chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.


ParsedChatCompletion[CityGuide](id='60nnaKHzGbKjvr0PppqLqQc', choices=[ParsedChoice[CityGuide](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[CityGuide](content='{\n  "is_acceptable": false\n}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, parsed=CityGuide(is_acceptable=False)))], created=1759988204, model='gemini-2.0-flash', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=151, total_tokens=162, completion_tokens_details=None, prompt_tokens_details=None))
ParsedChatCompletion[CityGuide](id='A0rnaM_AN6qh1e8P27HjeA', choices=[ParsedChoice[CityGuide](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[CityGuide](content='{\n  "is_acceptable": false\n}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, parsed=CityGuide(is_acceptable=False)))], c